In [1]:
%load_ext rpy2.ipython
import rpy2
import warnings
def rmagic_warning(
    message,
    category = rpy2.rinterface.RRuntimeWarning,
    filename = '',
    lineno = -1,
    file=None, 
    line=None):
    print(message)
default_showwarning = warnings.showwarning

In [2]:
from common_imports import *

# Process Affy data for PDXs and 7 primaries
These are the expression data used throughout the paper.

## Data provenance
On 3/28/17, Sebastian Brabetz (s.brabetz@dkfz-heidelberg.de) sent me through WeTransfer a zip archive including the tumor cels, cels for 7 primaries (it had been 8, but Med1612 was excluded because of contamination), and the most recent annotation Excel file, 170328_PDXScreen_Anno_v5.xlsx.

Most if not all of these files had been sent previously, spread over several occasions, and the annotation changed a couple of times as it was decided that e.g. one sample of a tumor was better than another (fewer passages etc). So I asked Sebastian to send everything fresh to start with a clean slate and make sure we were all using the same data.

## Process cel files to probewise abundance using hspeccdf

[Isella et al. 2015](https://www.ncbi.nlm.nih.gov/pubmed/25706627) showed that in order to reduce the influence of infiltration of mouse stroma on measuring gene expression in PDXs, it can help to remove probes that cross-hybridize to mouse transcripts. This is done using a custom content descriptor file (CDF) rather than the default hgu133plus2 CDF.

In [3]:
warnings.showwarning = rmagic_warning # to only print the warning text, not text + returned warning object
from rpy2.robjects import numpy2ri
numpy2ri.activate()

In [4]:
%%R
pkg.is.installed <- function(mypkg)
{
    return(mypkg %in% rownames(installed.packages()))
}

Note that some of the installation and configuration of these R packages may have to be done in the terminal. If the following cell does not work, try installing the packages manually and re-running the notebook.

In [6]:
%%R -i pdx_affy_cel_dir,pdx_affy_probe_exp_file
library('affy')
library('annotate')
library('hspeccdf')
print('Reading in cels')
data <- ReadAffy(celfile.path=pdx_affy_cel_dir, cdfname='hspeccdf')
print('Performing RMA')
result <- affy::rma(data)
write.exprs(result, pdx_affy_probe_exp_file)

Bioconductor version 3.6 (BiocInstaller 1.28.0), ?biocLite for help

A new version of Bioconductor is available after installing the most recent
  version of R; see http://bioconductor.org/install

Loading required package: BiocGenerics

Loading required package: parallel


Attaching package: 'BiocGenerics'


The following objects are masked from 'package:parallel':

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB


The following objects are masked from 'package:stats':

    IQR, mad, sd, var, xtabs


The following objects are masked from 'package:base':

    Filter, Find, Map, Position, Reduce, anyDuplicated, append,
    as.data.frame, cbind, colMeans, colSums, colnames, do.call,
    duplicated, eval, evalq, get, grep, grepl, intersect, is.unsorted,
    lapply, lengths, mapply, match, mget, order, paste, pmax, pmax.int,
    pmin, pmin.int, rank, rbind, rowMean

[1] "Reading in cels"
[1] "Performing RMA"
Background correcting
Normalizing
Calculating Expression


In [7]:
print('done!')

done!


In [8]:
numpy2ri.deactivate()
# Return to default warning handling.
warnings.showwarning = default_showwarning  

## Convert to genewise abundance

In [9]:
def fix_dkfz_tumor_name(name):
    name = name.replace('DKFZ-', '')
    name = name.upper()
    name = name.replace("FH", '')
    name = name.replace('-', '')
    name = re.sub('MB$', '', name)
    return name

In [15]:
pdx_annot_df = pd.read_excel(PDX_AFFY_ANNOT_FILE, index_col=0).astype(str)
cel_to_tumor = dict(zip(['{}.cel'.format(cel) for cel in pdx_annot_df.CEL_file], 
                        [fix_dkfz_tumor_name(name) for name in pdx_annot_df.index]))

have_primary = pdx_annot_df[pdx_annot_df.CEL_primary != '0']
cel_to_primary = dict(zip(['{}.cel'.format(cel) for cel in have_primary.CEL_primary], 
                        [fix_dkfz_tumor_name(name) for name in have_primary.index]))

Rather than using the hgu133plus2 annotation, hspeccdf maps probes to entrez genes, so for example the probeset 171484_at matches to the gene with entrez ID 171484. I use gene ID information from [HUGO](https://www.genenames.org/cgi-bin/download) (last downloaded 4.13.18) to map entrez IDs to HUGO gene symbols.

In [28]:
entrez2hugo = utils.make_col2col_dict(GENE_ID_CONVERSION_FILE, 'entrez', 'hugo')

In [29]:
probe_exp = pd.read_table(pdx_affy_probe_exp_file, index_col=0)
p2g = {}
for probe in probe_exp.index:
    entrez = probe.split('_')[0]
    if entrez in entrez2hugo:
        hugo = entrez2hugo[entrez]
        p2g[probe] = hugo
probe_df = probe_exp.copy()
keepers = []
for probe in probe_df.index:
    if probe in p2g:
        if not pd.isnull(p2g[probe]):
            keepers.append(probe)
gene_df = probe_df.loc[keepers]
gene_df.index = [p2g[probe] for probe in gene_df.index]

In [30]:
gene_df = utils.merge_redundant_series(gene_df, axis=0).T.sort_index(axis=1)

In [31]:
#gene_df = quantile_normalize(gene_df) # was done initially, but later considered unnecessary.

In [32]:
primaries_df = gene_df.loc[list(cel_to_primary.keys())]
primaries_df.index = [cel_to_primary[cel] for cel in primaries_df.index]
tumors_df = gene_df.loc[list(cel_to_tumor.keys())]
tumors_df.index = [cel_to_tumor[cel] for cel in tumors_df.index]

In [43]:
tumors_df.sort_index(axis=0).to_csv(PREPROC_PDX_AFFY_EXP_FILE)
primaries_df.sort_index(axis=0).to_csv(PREPROC_PRIMARY_AFFY_EXP_FILE)

In [44]:
tumors_df.shape

(20, 16247)

# Project expression onto genesets using ssGSEA

In [48]:
from genesets import load_genesets
from gsea import ssgsea

In [49]:
pdx_exp = pd.read_csv(PREPROC_PDX_AFFY_EXP_FILE, index_col=0)
primary_exp = pd.read_csv(PREPROC_PRIMARY_AFFY_EXP_FILE, index_col=0)

In [52]:
genesets = load_genesets(GENESETS_DIR, which='all')

In [53]:
%%time
pdx_ssgsea = ssgsea(pdx_exp, genesets, alpha=0.75)

CPU times: user 21.1 s, sys: 2.84 s, total: 23.9 s
Wall time: 8min 52s


In [56]:
pdx_ssgsea.to_csv(PDX_SSGSEA_FILE)

In [57]:
%%time
primary_ssgsea = ssgsea(primary_exp, genesets, alpha=0.75)

CPU times: user 7.47 s, sys: 890 ms, total: 8.36 s
Wall time: 3min 31s


In [59]:
primary_ssgsea.to_csv(PRIMARY_SSGSEA_FILE)

# Process Illumina Beadchip data for 3 PDXs and primaries

These data (available [here](https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE28192)) were not used in the paper but are included in the supplement and for reference.

In [60]:
from utils import merge_redundant_series

In [65]:
exp_matrix = pd.read_table(PDX_PRIMARY_BEADCHIP_SERIES_MATRIX_FILE, comment='!', index_col=0)
exp_matrix.head()

,GSM697986,GSM697987,GSM697988,GSM697989,GSM697990,GSM697991,GSM697992,GSM697993,GSM697994,GSM697995,...,GSM698068,GSM698069,GSM698070,GSM698071,GSM698072,GSM698073,GSM698074,GSM698075,GSM698076,GSM698077
ID_REF,,,,,,,,,,,,,,,,,,,,,
ILMN_1343289,8873.765,9019.939,8321.701,8597.173,7972.136,8454.363,7222.372,7160.046,5800.359,5789.193,...,3966.823,4510.685,7933.080,7655.048,3969.261,4202.289,6584.224,6251.000,7800.057,7169.415
ILMN_1343290,10581.030,11269.900,3117.435,3319.712,8648.661,8912.334,16107.970,16039.770,14342.820,15226.370,...,6941.944,6465.565,5421.817,5232.795,8435.935,8016.704,10082.870,9275.180,4842.793,5161.231
ILMN_1343291,27723.540,26947.170,25108.230,24472.990,23876.240,25107.190,30136.330,31310.060,24931.080,26643.790,...,13985.140,15141.920,21393.430,23346.100,19563.940,20737.940,21042.820,20138.190,22116.220,24224.900
ILMN_1343292,3720.148,3716.892,3045.436,3093.125,3431.755,3183.872,6163.287,5984.875,6520.189,6664.529,...,7127.288,7485.877,10175.230,11400.910,6385.931,6290.254,7631.206,7654.023,10857.410,10923.240
ILMN_1343293,8413.422,8265.768,5738.696,5639.104,9057.703,9153.695,4510.629,4578.813,5262.731,5183.555,...,3803.817,3709.520,3953.159,4444.244,2922.038,2886.281,2843.153,2871.426,3956.004,4251.552


In [66]:
ilmn_id_to_tumor = {"GSM698018": "Pt-1299",
"GSM698019": "Pt-1299",
"GSM698020": "ICb1299",
"GSM698021": "ICb1299",
"GSM698032": "Pt-1487",
"GSM698033": "Pt-1487",
"GSM698034": "ICb-1487",
"GSM698035": "ICb-1487",
"GSM698046": "Pt-1572",
"GSM698047": "Pt-1572",
"GSM698048": "ICb-1572",
"GSM698049": "ICb-1572"}

In [67]:
exp_df = exp_matrix.loc[:, ilmn_id_to_tumor.keys()]
exp_df.columns = [ilmn_id_to_tumor[ilmn_id] for ilmn_id in exp_df.columns]
exp_df = merge_redundant_series(exp_df, axis=1)

In [70]:
# downloaded from GEO's page for the platform
ilmn_annot = pd.read_table(PDX_PRIMARY_BEADCHIP_ILMN_ANNOT_FILE, comment='#', index_col=0).Symbol
ilmn_annot.head()

ID
ILMN_1725881     LOC23117
ILMN_1910180          NaN
ILMN_1804174       FCGR2B
ILMN_1810835        SPRR3
ILMN_1758197    LOC653895
Name: Symbol, dtype: object

In [71]:
nonan = ilmn_annot.dropna()
nonan.head()

ID
ILMN_1725881     LOC23117
ILMN_1804174       FCGR2B
ILMN_1810835        SPRR3
ILMN_1758197    LOC653895
ILMN_1668162      DGAT2L3
Name: Symbol, dtype: object

In [72]:
exp_df = exp_df.loc[nonan.index]
exp_df.index = [nonan.loc[idx] for idx in exp_df.index]
exp_df = merge_redundant_series(exp_df, axis=0).T.sort_index(axis=1)
exp_df.shape

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  """Entry point for launching an IPython kernel.


(6, 24614)

In [73]:
def rename_icb_samples(df):
    newdf = df.copy()
    newdf.index = [re.sub('.*?([0-9].+).*?', r"ICB\1", s) for s in df.index]
    return newdf

In [74]:
primaries = [idx for idx in exp_df.index if idx.startswith('Pt')]
primary_exp = rename_icb_samples(exp_df.loc[primaries].sort_index(axis=0))
tumor_exp = rename_icb_samples(exp_df.drop(primaries, axis=0).sort_index(axis=0))

In [79]:
tumor_exp.to_csv(PREPROC_PDX_BEADCHIP_EXP_FILE)
primary_exp.to_csv(PREPROC_PRIMARY_BEADCHIP_EXP_FILE)

# Write expression to supplementary file

I am actually not sure if we'll need this supplement, because the expression will be going in GEO.

In [82]:
bc_pdx = pd.read_csv(PREPROC_PDX_BEADCHIP_EXP_FILE, index_col=0).sort_index(axis=1)
bc_pt = pd.read_csv(PREPROC_PRIMARY_BEADCHIP_EXP_FILE, index_col=0).sort_index(axis=1)
pdx_exp = pd.read_csv(PREPROC_PDX_AFFY_EXP_FILE, index_col=0)
pdx_primary_exp = pd.read_csv(PREPROC_PRIMARY_AFFY_EXP_FILE, index_col=0)

In [ ]:
# This cell was not run! 2018-12-13

In [83]:
from modules.utils import save_xls
sheetnames = 'affy_pdx affy_primary illumina_pdx illumina_primary'.split()
dfs = [pdx_exp, pdx_primary_exp, bc_pdx, bc_pt]
save_xls(dfs, sheetnames, 'SUPPL_PREPROC_EXP_FILE')

ValueError: No engine for filetype: ''